### ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) **Delta External tables**

#### **Crear Database**

In [ ]:
CREATE DATABASE IF NOT EXISTS f1_demo
LOCATION '/mnt/formula1dl/demo'

#### **Leer archivo en un dataframe**

In [ ]:
results_df = spark.read \
.option("inferSchema", True) \
.json("/mnt/formula1dl/raw/2021-03-28/results.json")

#### **Crear DELTA EXTERNAL TABLES**

- Recordar que para las **EXTERNAL TABLES** debemos eliminar la tabla y el directorio y archivo por separado

##### **Forma 1**

In [ ]:
results_df.write.mode("overwrite").save("/mnt/formula1dl/demo/results_external")

<center><img src="https://i.postimg.cc/vT5wbjJT/db167.png"></center>

In [ ]:
-- Debemos crear la tabla utilizando Databricks SQL
CREATE TABLE f1_demo.results_external
LOCATION '/mnt/formula1dl/demo/results_external'

-- O podemos utilizar la siguiente forma
CREATE TABLE f1_demo.results_external
USING DELTA
LOCATION '/mnt/formula1dl/demo/results_external'

In [ ]:
DESCRIBE FORMATTED f1_demo.results_external;

col_name,data_type,comment
constructorId,bigint,null
driverId,bigint,null
fastestLap,string,null
fastestLapSpeed,string,null
fastestLapTime,string,null
grid,bigint,null
laps,bigint,null
milliseconds,string,null
number,bigint,null
points,bigint,null


In [ ]:
SELECT * FROM f1_demo.results_external;

constructorId,driverId,fastestLap,fastestLapSpeed,fastestLapTime,grid,laps,milliseconds,number,points,position,positionOrder,positionText,raceId,rank,resultId,statusId,time
131,1,44,207.235,1:34.015,2,56,5523897,44,25,1,1,1,1052,4,24966,1,1:32:03.897
9,830,41,208.984,1:33.228,1,56,5524642,33,18,2,2,2,1052,2,24967,1,+0.745
131,822,56,211.566,1:32.090,3,56,5561280,77,16,3,3,3,1052,1,24968,1,+37.383
1,846,38,206.398,1:34.396,7,56,5570363,4,12,4,4,4,1052,6,24969,1,+46.466
9,815,44,207.334,1:33.970,0,56,5575944,11,10,5,5,5,1052,3,24970,1,+52.047
6,844,39,205.112,1:34.988,4,56,5582987,16,8,6,6,6,1052,11,24971,1,+59.090
1,817,36,205.233,1:34.932,6,56,5589901,3,6,7,7,7,1052,10,24972,1,+66.004
6,832,48,206.151,1:34.509,8,56,5590997,55,4,8,8,8,1052,7,24973,1,+67.100
213,852,38,205.603,1:34.761,13,56,5609589,22,2,9,9,9,1052,8,24974,1,+85.692
117,840,31,205.378,1:34.865,10,56,5610610,18,1,10,10,10,1052,9,24975,1,+86.713


In [ ]:
-- Eliminamos la tabla
DROP TABLE f1_demo.results_external

In [ ]:
# Eliminamos el directorio y sus archivos
dbutils.fs.rm("/mnt/formula1dl/demo/results_external", True)

Out[39]: True

##### Forma 2

In [ ]:
results_df.write.format("delta").mode("overwrite").save("/mnt/formula1dl/demo/results_external")

<center><img src="https://i.postimg.cc/1XZSbs84/db169.png"></center>

In [ ]:
results_external_df = spark.read.format("delta").load("/mnt/formula1dl/demo/results_external")

In [ ]:
results_external_df.show(truncate=False)

+-------------+--------+----------+---------------+--------------+----+----+------------+------+------+--------+-------------+------------+------+----+--------+--------+-----------+
|constructorId|driverId|fastestLap|fastestLapSpeed|fastestLapTime|grid|laps|milliseconds|number|points|position|positionOrder|positionText|raceId|rank|resultId|statusId|time       |
+-------------+--------+----------+---------------+--------------+----+----+------------+------+------+--------+-------------+------------+------+----+--------+--------+-----------+
|131          |1       |44        |207.235        |1:34.015      |2   |56  |5523897     |44    |25    |1       |1            |1           |1052  |4   |24966   |1       |1:32:03.897|
|9            |830     |41        |208.984        |1:33.228      |1   |56  |5524642     |33    |18    |2       |2            |2           |1052  |2   |24967   |1       |+0.745     |
|131          |822     |56        |211.566        |1:32.090      |3   |56  |5561280     |7

In [ ]:
CREATE TABLE f1_demo.results_external
LOCATION '/mnt/formula1dl/demo/results_external'

In [ ]:
DESCRIBE FORMATTED f1_demo.results_external;

col_name,data_type,comment
constructorId,bigint,null
driverId,bigint,null
fastestLap,string,null
fastestLapSpeed,string,null
fastestLapTime,string,null
grid,bigint,null
laps,bigint,null
milliseconds,string,null
number,bigint,null
points,bigint,null


In [ ]:
-- Eliminamos la tabla
DROP TABLE f1_demo.results_external

In [ ]:
# Eliminamos el directorio y sus archivos
dbutils.fs.rm("/mnt/formula1dl/demo/results_external", True)

Out[47]: True